### 加载数据

In [1]:
from openpyxl import Workbook
import os
import math

base_dir = r'D:\项目资料\基因表达\20220804'
genes_file = os.path.join(base_dir, 'Diff_genes_name.csv')
patients_file = os.path.join(base_dir, '最终纳入的患者.csv')
m0_genes_file = os.path.join(base_dir, 'm0_all.csv')
m1_genes_file = os.path.join(base_dir, 'm1_all.csv')

# 400多个基因
gene_dict = {}
with open(genes_file) as f:
    for line in f.readlines()[1:]:
        arr = line.strip().split(',')
        gene_dict[arr[1]] = arr[2]
gene_id_arr = [gene_id for gene_id in gene_dict.keys()]
gene_num = len(gene_id_arr)
# print(gene_dict)

# 加载实验对象
ensemble_m0, ensemble_m1 = [], []
with open(patients_file, encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        arr = line.strip().split(',')
        if arr[2] == 'M0':
            ensemble_m0.append(arr[0])
        else:
            ensemble_m1.append(arr[0])
# print(ensemble_m0, ensemble_m1)


### 加载并过滤基因数据

In [2]:
# M0
gene_data_m0 = {}
with open(m0_genes_file) as f:
    for idx, line in enumerate(f.readlines()):
        arr = line.strip().split(',')
        if idx == 0:
            for name in arr[1:]:
                patient_id = name[:12]
                if patient_id in ensemble_m0:
                    gene_data_m0[(patient_id, arr.index(name))] = [0 for i in range(len(gene_id_arr))]
        else:
            gene_id = arr[0].split('.')[0]
            if gene_id in gene_dict:
                gene_id_idx = gene_id_arr.index(gene_id)
                for patient_id, pidx in gene_data_m0.keys():
                    gene_data_m0[(patient_id, pidx)][gene_id_idx] = math.log(float(arr[pidx]) + 1, 10)

# M1
gene_data_m1 = {}
with open(m1_genes_file) as f:
    for idx, line in enumerate(f.readlines()):
        arr = line.strip().split(',')
        if idx == 0:
            for name in arr[1:]:
                patient_id = name[:12]
                if patient_id in ensemble_m1:
                    gene_data_m1[(patient_id, arr.index(name))] = [0 for i in range(len(gene_id_arr))]
        else:
            gene_id = arr[0].split('.')[0]
            if gene_id in gene_dict:
                gene_id_idx = gene_id_arr.index(gene_id)
                for patient_id, pidx in gene_data_m1.keys():
                    gene_data_m1[(patient_id, pidx)][gene_id_idx] = math.log(float(arr[pidx]) + 1, 10)


gene_data = {'M0': {}, 'M1': {}}
for patient_id, pidx in gene_data_m0.keys():
    gene_data['M0'][patient_id] = gene_data_m0[(patient_id, pidx)]
for patient_id, pidx in gene_data_m1.keys():
    gene_data['M1'][patient_id] = gene_data_m1[(patient_id, pidx)]

print('基因字典：%s, M0: %s, M1: %s' % (len(gene_dict), len(gene_data['M0']), len(gene_data['M1'])))


基因字典：428, M0: 326, M1: 61


### 基因预测模型定义

In [3]:
import numpy as np
import pandas as pd
import os
import time
import tensorflow as tf
from tensorflow.keras import layers, datasets
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD

In [4]:
hidden_size = 1024

def build_model():
    #     scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    
    inputs = tf.keras.Input(shape=(2048,))
    
    # 模型定义
    #     x = scale_layer(inputs)
    x = inputs
    x = tf.keras.layers.Dense(1024)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # Regularize with dropout
    x = tf.keras.layers.Dense(512)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # Regularize with dropout
    x = tf.keras.layers.Dense(gene_num)(x)
    
    #     outputs = tf.keras.layers.Softmax()(x)
    outputs = x
    model = tf.keras.Model(inputs, outputs)
    
    model.summary()
    
    return model

In [5]:
# cache path
cache_path = r'D:\projects\Vision\GenesPred\ckpts'
if not os.path.exists(cache_path):
    os.mkdir(cache_path)
    
def get_mdl_name():
    save_path = cache_path
    mdl_save_name = 'mdl_' + time.strftime("%Y%m%d", time.localtime(time.time())) + '_{epoch:02d}-{val_loss:.4f}.hdf5'
    abs_model_name = os.path.join(save_path, mdl_save_name)
    
    return abs_model_name

In [6]:
def scheduler(epoch, lr):
    if epoch > 2:
        lr = lr * tf.math.exp(-0.051)
    return lr
    
early_stop = EarlyStopping(monitor='val_loss', patience=3)
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

def get_callbacks(unfrozen_layers=None):
    model_checkpoint = ModelCheckpoint(get_mdl_name(), monitor='val_loss', verbose=2, save_best_only=True)
    callbacks = [early_stop, model_checkpoint, scheduler_callback]
    return callbacks

In [7]:
scheduler(21, 0.01)

<tf.Tensor: shape=(), dtype=float32, numpy=0.009502786>

In [8]:
# 定义模型
model = build_model()
model.compile(optimizer = Adam(learning_rate = 0.0001), 
              loss = tf.keras.losses.MeanSquaredError(), 
              metrics = ['mse'])


# # 训练
# history = model.fit(train_gen, steps_per_epoch=train_gen.n // BATCH_SIZE, epochs=epochs,
#                     validation_data=val_gen, validation_steps=val_gen.n // BATCH_SIZE,
#                     callbacks=get_callbacks(modl_type, data_type), workers=1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2048)]            0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 1024)              0         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                             

### 加载特征数据

In [9]:
import os

def get_features(file_path):
    features = []
    with open(file_path) as f:
        for line in f.readlines():
            features.append([float(e) for e in line.split(' ')])
    return features

def gen_tensor_from_path(feature_path, batch_size):
    features, labels = [], []
    for subfolder in ['M0', 'M1']:
        print('subfolder: %s' % subfolder)
        folder = os.path.join(feature_path, subfolder)
        for file_name in os.listdir(folder):
            print('file_name: %s' % file_name)
            patient_id = file_name[:12]
            if patient_id not in gene_data[subfolder]:
                continue

            file_path = os.path.join(folder, file_name)
            features_ = get_features(file_path)
            features.extend(features_)

            gene_arr = gene_data[subfolder][patient_id]
            labels.extend([gene_arr for i in range(len(features_))])
            
            if len(features) >= batch_size:
                rt_features, rt_labels = features[:batch_size], labels[:batch_size]
                features, labels = features[batch_size:], labels[batch_size:]
                yield rt_features, rt_labels
                
#     print("特征维度：%s, 标签维度：%s" % (len(features), len(labels)))

def data_generator(feature_path):
    for subfolder in ['M0', 'M1']:
        folder = os.path.join(feature_path, subfolder)
        for file_name in os.listdir(folder):
            patient_id = file_name[:12]
            if patient_id not in gene_data[subfolder]:
                continue

            file_path = os.path.join(folder, file_name)
            features = get_features(file_path)
            gene_arr = gene_data[subfolder][patient_id]
            labels = [gene_arr for i in range(len(features))]
            
            for feature, label in zip(features, labels):
                yield feature, label

In [10]:
feature_base_path = r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727特征'
train_feature_path = os.path.join(feature_base_path, 'train')
val_feature_path = os.path.join(feature_base_path, 'test')

BATCH_SIZE = 8000
epochs = 100

In [11]:
# # 训练集数据
# print('处理文件夹：%s' % train_feature_path)
# train_data = gen_tensor_from_path(train_feature_path)
        
# # 验证集数据
# print('处理文件夹：%s' % test_feature_path)
# val_data = gen_tensor_from_path(test_feature_path)


train_dataset = tf.data.Dataset.from_generator(lambda: data_generator(train_feature_path), 
                                               output_types=(tf.float32, tf.float32)).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_generator(lambda: data_generator(val_feature_path), 
                                               output_types=(tf.float32, tf.float32)).batch(BATCH_SIZE)

# tpath = r'D:\项目资料\基因表达\数据\特征'
# train_dataset = tf.data.Dataset.from_generator(lambda: data_generator(tpath), 
#                                                output_types=(tf.float32, tf.float32)).batch(BATCH_SIZE)

# val_dataset = tf.data.Dataset.from_generator(lambda: data_generator(tpath), 
#                                                output_types=(tf.float32, tf.float32)).batch(BATCH_SIZE)

In [13]:
list(train_dataset.take(1))

[(<tf.Tensor: shape=(8000, 2048), dtype=float32, numpy=
  array([[0.        , 0.        , 0.03479557, ..., 2.4269228 , 0.        ,
          0.        ],
         [0.        , 0.        , 0.02271368, ..., 1.8509197 , 0.        ,
          0.        ],
         [0.        , 0.        , 0.00568234, ..., 2.9525747 , 0.        ,
          0.        ],
         ...,
         [0.        , 0.        , 0.0264525 , ..., 2.8940635 , 0.        ,
          0.        ],
         [0.        , 0.        , 0.05987207, ..., 2.7817323 , 0.        ,
          0.        ],
         [0.        , 0.        , 0.15166526, ..., 3.0097744 , 0.        ,
          0.        ]], dtype=float32)>,
  <tf.Tensor: shape=(8000, 428), dtype=float32, numpy=
  array([[2.178977  , 2.4727564 , 1.9344984 , ..., 0.47712126, 0.30103   ,
          0.60206   ],
         [2.178977  , 2.4727564 , 1.9344984 , ..., 0.47712126, 0.30103   ,
          0.60206   ],
         [2.178977  , 2.4727564 , 1.9344984 , ..., 0.47712126, 0.30103   

In [12]:
# history = model.fit(train_dataset, steps_per_epoch=len(train_features) // BATCH_SIZE, epochs=epochs,
#                     validation_data=val_dataset, validation_steps=len(val_features) // BATCH_SIZE,
#                     callbacks=get_callbacks(), workers=1)

history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=get_callbacks(), workers=1)

Epoch 1/100
    183/Unknown - 1171s 6s/step - loss: 1.0530 - mse: 1.0530
Epoch 1: val_loss improved from inf to 0.47743, saving model to D:\projects\Vision\GenesPred\ckpts\mdl_20220808_01-0.4774.hdf5
183/183 [==============================] - 1452s 8s/step - loss: 1.0530 - mse: 1.0530 - val_loss: 0.4774 - val_mse: 0.4774 - lr: 1.0000e-04
Epoch 2/100
183/183 [==============================] - ETA: 0s - loss: 0.5836 - mse: 0.5836
Epoch 2: val_loss improved from 0.47743 to 0.38226, saving model to D:\projects\Vision\GenesPred\ckpts\mdl_20220808_02-0.3823.hdf5
183/183 [==============================] - 1412s 8s/step - loss: 0.5836 - mse: 0.5836 - val_loss: 0.3823 - val_mse: 0.3823 - lr: 1.0000e-04
Epoch 3/100
183/183 [==============================] - ETA: 0s - loss: 0.5111 - mse: 0.5111
Epoch 3: val_loss improved from 0.38226 to 0.37107, saving model to D:\projects\Vision\GenesPred\ckpts\mdl_20220808_03-0.3711.hdf5
183/183 [==============================] - 1411s 8s/step - loss: 0.5111 - 

### 使用MLP预测，然后合并结果

In [13]:
mdl_path = r'ckpts\mdl_20220808_09-0.3394.hdf5'
new_model = tf.keras.models.load_model(mdl_path)
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2048)]            0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 1024)              0         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                             

In [47]:
def agregate(x, agg_type):
    x = np.array(x)
    if agg_type == 'max':
        return np.max(x, axis=0)
    elif agg_type == 'mean':
        return np.mean(x, axis=0)
    elif agg_type == 'min':
        return np.min(x, axis=0)
    elif agg_type == 'quantile':
        return np.quantile(x, 0.75, interpolation='higher', axis=0)
    
    
def pred_from_path(feature_path, agg_type):
    labels_arr, pred_arr = [], []
    for subfolder in ['M0', 'M1']:
        print('subfolder: %s' % subfolder)
        folder = os.path.join(feature_path, subfolder)
        for file_name in os.listdir(folder):
            print('file_name: %s' % file_name)
            patient_id = file_name[:12]
            if patient_id not in gene_data[subfolder]:
                continue

            file_path = os.path.join(folder, file_name)
            features = get_features(file_path)
            pred = model.predict(tf.convert_to_tensor(features), batch_size=len(features), verbose=1)
            pred_agg = agregate(pred, agg_type)
            print(pred_agg)
            pred_arr.append(pred_agg)

            gene_arr = gene_data[subfolder][patient_id]
            labels_arr.append(gene_arr)
    return pred_arr, labels_arr

In [15]:
pred_arr, labels_arr = pred_from_path(val_feature_path, 'max')

subfolder: M0
file_name: TCGA-A6-2672-01Z-00-DX1.e2a845c8-6d77-4120-9f43-abec84a66c1c.txt
1/1 [==============================] - 0s 157ms/step
file_name: TCGA-A6-2680-01Z-00-DX1.7b77c0fb-f51d-4d16-ae77-f7615b1d0b87.txt
1/1 [==============================] - 0s 70ms/step
file_name: TCGA-A6-3809-01Z-00-DX1.c26f03e8-c285-4a66-925d-ae9cba17d7b3.txt
1/1 [==============================] - 0s 27ms/step
file_name: TCGA-A6-3810-01Z-00-DX1.2940ca70-013a-4bc3-ad6a-cf4d9ffa77ce.txt
1/1 [==============================] - 0s 48ms/step
file_name: TCGA-A6-4105-01Z-00-DX1.228b02a5-04fa-4392-bf03-b297c19665c3.txt
1/1 [==============================] - 0s 48ms/step
file_name: TCGA-A6-5661-01Z-00-DX1.bad2d858-11b4-4b9c-a720-daaae592cf48.txt
1/1 [==============================] - 0s 25ms/step
file_name: TCGA-A6-6140-01Z-00-DX1.f34d99be-25dd-4811-9155-0dbb53e849ac.txt
1/1 [==============================] - 0s 32ms/step
file_name: TCGA-A6-6654-01Z-00-DX1.ed491b61-7c44-4275-879b-22f8007b5ff1.txt
1/1 [========

1/1 [==============================] - 0s 48ms/step
file_name: TCGA-QG-A5Z2-01Z-00-DX1.51896C31-235E-48EF-90F7-FC05350CA564.txt
1/1 [==============================] - 0s 49ms/step
subfolder: M1
file_name: TCGA-5M-AAT6-01Z-00-DX1.8834C952-14E3-4491-8156-52FC917BB014.txt
1/1 [==============================] - 0s 50ms/step
file_name: TCGA-A6-2674-01Z-00-DX1.d301f1f5-6f4a-49e6-9c93-f4e8b7f616b8.txt
1/1 [==============================] - 0s 18ms/step
file_name: TCGA-A6-2682-01Z-00-DX1.be71dca0-b9b7-40be-a6c6-9d053c7886a6.txt
1/1 [==============================] - 0s 33ms/step
file_name: TCGA-A6-5662-01Z-00-DX1.82569684-1c31-4346-af9b-c296a020f624.txt
1/1 [==============================] - 0s 20ms/step
file_name: TCGA-A6-6142-01Z-00-DX1.e923ce20-d3c3-4d21-9e7c-d999a3742f9b.txt
1/1 [==============================] - 0s 26ms/step
file_name: TCGA-A6-6652-01Z-00-DX1.30916007-088e-48bd-abf8-519f34e2c37a.txt
1/1 [==============================] - 0s 18ms/step
file_name: TCGA-A6-A567-01Z-00-DX1.F94

In [24]:
pred_min_arr, labels_arr = pred_from_path(val_feature_path, 'min')

subfolder: M0
file_name: TCGA-A6-2672-01Z-00-DX1.e2a845c8-6d77-4120-9f43-abec84a66c1c.txt
1/1 [==============================] - 0s 29ms/step
file_name: TCGA-A6-2680-01Z-00-DX1.7b77c0fb-f51d-4d16-ae77-f7615b1d0b87.txt
1/1 [==============================] - 0s 31ms/step
file_name: TCGA-A6-3809-01Z-00-DX1.c26f03e8-c285-4a66-925d-ae9cba17d7b3.txt
1/1 [==============================] - 0s 25ms/step
file_name: TCGA-A6-3810-01Z-00-DX1.2940ca70-013a-4bc3-ad6a-cf4d9ffa77ce.txt
1/1 [==============================] - 0s 18ms/step
file_name: TCGA-A6-4105-01Z-00-DX1.228b02a5-04fa-4392-bf03-b297c19665c3.txt
1/1 [==============================] - 0s 54ms/step
file_name: TCGA-A6-5661-01Z-00-DX1.bad2d858-11b4-4b9c-a720-daaae592cf48.txt
1/1 [==============================] - 0s 24ms/step
file_name: TCGA-A6-6140-01Z-00-DX1.f34d99be-25dd-4811-9155-0dbb53e849ac.txt
1/1 [==============================] - 0s 33ms/step
file_name: TCGA-A6-6654-01Z-00-DX1.ed491b61-7c44-4275-879b-22f8007b5ff1.txt
1/1 [=========

1/1 [==============================] - 0s 50ms/step
file_name: TCGA-QG-A5Z2-01Z-00-DX1.51896C31-235E-48EF-90F7-FC05350CA564.txt
1/1 [==============================] - 0s 50ms/step
subfolder: M1
file_name: TCGA-5M-AAT6-01Z-00-DX1.8834C952-14E3-4491-8156-52FC917BB014.txt
1/1 [==============================] - 0s 49ms/step
file_name: TCGA-A6-2674-01Z-00-DX1.d301f1f5-6f4a-49e6-9c93-f4e8b7f616b8.txt
1/1 [==============================] - 0s 17ms/step
file_name: TCGA-A6-2682-01Z-00-DX1.be71dca0-b9b7-40be-a6c6-9d053c7886a6.txt
1/1 [==============================] - 0s 34ms/step
file_name: TCGA-A6-5662-01Z-00-DX1.82569684-1c31-4346-af9b-c296a020f624.txt
1/1 [==============================] - 0s 21ms/step
file_name: TCGA-A6-6142-01Z-00-DX1.e923ce20-d3c3-4d21-9e7c-d999a3742f9b.txt
1/1 [==============================] - 0s 27ms/step
file_name: TCGA-A6-6652-01Z-00-DX1.30916007-088e-48bd-abf8-519f34e2c37a.txt
1/1 [==============================] - 0s 18ms/step
file_name: TCGA-A6-A567-01Z-00-DX1.F94

In [25]:
pred_mean_arr, labels_arr = pred_from_path(val_feature_path, 'mean')

subfolder: M0
file_name: TCGA-A6-2672-01Z-00-DX1.e2a845c8-6d77-4120-9f43-abec84a66c1c.txt
1/1 [==============================] - 0s 26ms/step
file_name: TCGA-A6-2680-01Z-00-DX1.7b77c0fb-f51d-4d16-ae77-f7615b1d0b87.txt
1/1 [==============================] - 0s 31ms/step
file_name: TCGA-A6-3809-01Z-00-DX1.c26f03e8-c285-4a66-925d-ae9cba17d7b3.txt
1/1 [==============================] - 0s 25ms/step
file_name: TCGA-A6-3810-01Z-00-DX1.2940ca70-013a-4bc3-ad6a-cf4d9ffa77ce.txt
1/1 [==============================] - 0s 17ms/step
file_name: TCGA-A6-4105-01Z-00-DX1.228b02a5-04fa-4392-bf03-b297c19665c3.txt
1/1 [==============================] - 0s 49ms/step
file_name: TCGA-A6-5661-01Z-00-DX1.bad2d858-11b4-4b9c-a720-daaae592cf48.txt
1/1 [==============================] - 0s 25ms/step
file_name: TCGA-A6-6140-01Z-00-DX1.f34d99be-25dd-4811-9155-0dbb53e849ac.txt
1/1 [==============================] - 0s 30ms/step
file_name: TCGA-A6-6654-01Z-00-DX1.ed491b61-7c44-4275-879b-22f8007b5ff1.txt
1/1 [=========

1/1 [==============================] - 0s 50ms/step
file_name: TCGA-QG-A5Z2-01Z-00-DX1.51896C31-235E-48EF-90F7-FC05350CA564.txt
1/1 [==============================] - 0s 49ms/step
subfolder: M1
file_name: TCGA-5M-AAT6-01Z-00-DX1.8834C952-14E3-4491-8156-52FC917BB014.txt
1/1 [==============================] - 0s 51ms/step
file_name: TCGA-A6-2674-01Z-00-DX1.d301f1f5-6f4a-49e6-9c93-f4e8b7f616b8.txt
1/1 [==============================] - 0s 17ms/step
file_name: TCGA-A6-2682-01Z-00-DX1.be71dca0-b9b7-40be-a6c6-9d053c7886a6.txt
1/1 [==============================] - 0s 33ms/step
file_name: TCGA-A6-5662-01Z-00-DX1.82569684-1c31-4346-af9b-c296a020f624.txt
1/1 [==============================] - 0s 21ms/step
file_name: TCGA-A6-6142-01Z-00-DX1.e923ce20-d3c3-4d21-9e7c-d999a3742f9b.txt
1/1 [==============================] - 0s 26ms/step
file_name: TCGA-A6-6652-01Z-00-DX1.30916007-088e-48bd-abf8-519f34e2c37a.txt
1/1 [==============================] - 0s 18ms/step
file_name: TCGA-A6-A567-01Z-00-DX1.F94

In [48]:
pred_quantile_arr, labels_arr = pred_from_path(val_feature_path, 'quantile')

subfolder: M0
file_name: TCGA-A6-2672-01Z-00-DX1.e2a845c8-6d77-4120-9f43-abec84a66c1c.txt
1/1 [==============================] - 0s 25ms/step
[3.1388817  1.9833478  1.5592271  0.833777   1.081009   1.8128102
 1.3406107  1.621324   0.75396514 0.6613574  0.8329112  0.80524814
 0.42576164 0.8832969  0.63245726 0.15594183 1.2219777  1.3394029
 1.488549   1.7677368  0.64321655 1.5793631  0.34034798 1.1207744
 1.1726085  0.13316995 1.0650471  1.4479748  0.8656991  0.2309666
 0.8457025  0.8986153  0.11051849 2.2595265  0.23998618 0.964398
 1.2366542  1.2080649  0.7192571  2.9902325  0.35699332 2.2335181
 1.3529271  3.6860092  0.14543979 0.3381651  0.35301453 0.840936
 2.8243861  0.88957596 0.97732687 1.5658169  0.86480737 1.9469458
 1.5999271  1.3240744  0.89346296 1.4226272  1.2859391  1.3420837
 0.5312473  0.608354   0.9044878  2.5061467  0.7844618  1.514218
 1.5719374  3.5586772  1.5382621  0.28146112 1.0453405  0.35103902
 1.4190261  0.7298271  2.9236155  2.5140793  0.46668756 1.1333053
 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_33256\3920218272.py:10: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  return np.quantile(x, 0.75, interpolation='higher', axis=0)


1/1 [==============================] - 0s 30ms/step
[3.0315049  1.9143863  1.5114338  0.8006678  1.0500289  1.742661
 1.2972265  1.5809244  0.7123508  0.6411084  0.8094572  0.76398253
 0.40823343 0.84660363 0.5994018  0.14680095 1.1779087  1.2956967
 1.4282819  1.7052425  0.6279489  1.5148902  0.3349494  1.0717516
 1.1271333  0.12000293 1.0268838  1.3976446  0.8323806  0.21400262
 0.8119564  0.86289054 0.11238807 2.167892   0.233413   0.92020607
 1.1911958  1.1517375  0.7001821  2.8606825  0.34496525 2.1654308
 1.3053551  3.5260713  0.15081802 0.32765797 0.34712887 0.80272985
 2.7139688  0.8726093  0.9498882  1.4935362  0.8364512  1.8715315
 1.5362054  1.2776386  0.85468    1.3705349  1.2466015  1.2946312
 0.51561874 0.58118427 0.8612589  2.419802   0.7655389  1.4511873
 1.5214907  3.398477   1.4796617  0.27622658 0.9831466  0.35362312
 1.3744111  0.69852453 2.8121455  2.423516   0.45203283 1.0914333
 1.7760901  1.2327967  0.7261279  0.34579518 0.6663814  0.924352
 0.13822316 1.1495361

1/1 [==============================] - 0s 18ms/step
[3.2216167  2.0272455  1.6112851  0.8703618  1.1108732  1.8653419
 1.3928671  1.6674541  0.768717   0.70333284 0.8717993  0.8160438
 0.44941208 0.90429586 0.6368585  0.16378297 1.2659632  1.3706208
 1.5215375  1.8264635  0.67603505 1.6322608  0.36401105 1.143899
 1.2033905  0.15225627 1.091407   1.4897108  0.88587344 0.23071037
 0.8748973  0.92795604 0.11118452 2.341527   0.26317924 0.99391776
 1.274205   1.2255036  0.7736352  3.0375137  0.36733973 2.3020916
 1.378227   3.7903037  0.14721876 0.34682322 0.36139774 0.8580567
 2.918835   0.9352539  1.0018955  1.6125784  0.90301216 1.9899681
 1.6508635  1.3651665  0.91176164 1.4791774  1.3304294  1.3791995
 0.54968023 0.6331657  0.9158173  2.566594   0.81165403 1.5599389
 1.6360224  3.6489515  1.5904582  0.29022276 1.0838947  0.3582441
 1.4743929  0.75711113 2.9866745  2.5719843  0.49366045 1.1592692
 1.8847753  1.3161958  0.77891177 0.384946   0.7006639  0.99129885
 0.16524336 1.234446  

1/1 [==============================] - 0s 25ms/step
[3.2668576  2.0680811  1.6267715  0.87863874 1.1331636  1.8887693
 1.4299469  1.6969668  0.78388244 0.71243095 0.87607884 0.82770276
 0.45343375 0.917722   0.6552836  0.16788803 1.2834288  1.3999715
 1.5454966  1.8552812  0.6889915  1.6486171  0.3782586  1.16293
 1.2227064  0.1629946  1.1161711  1.5108451  0.90788215 0.23706657
 0.8945125  0.9491156  0.11534284 2.3761985  0.2689318  1.007676
 1.2963768  1.248248   0.7773807  3.0817347  0.3783171  2.3285375
 1.4077798  3.840608   0.15461205 0.36215162 0.36178133 0.8734341
 2.9580374  0.93721426 1.0226939  1.6431894  0.91266453 2.0199258
 1.6644052  1.382268   0.92520165 1.4920274  1.3448249  1.3990974
 0.5600704  0.6384568  0.947316   2.6100507  0.8222139  1.5782169
 1.6565022  3.6953282  1.6126469  0.30657113 1.1015795  0.36150917
 1.4919515  0.76432574 3.0327213  2.616113   0.49741557 1.1844469
 1.9114228  1.3358836  0.78814435 0.3871458  0.71896446 1.0018853
 0.17741533 1.2674987  1

...

1/1 [==============================] - 0s 26ms/step
[3.2724886  2.0686462  1.6342425  0.8797347  1.1300241  1.8978311
 1.4303061  1.6913261  0.7877391  0.718804   0.8843954  0.828187
 0.45535377 0.9177367  0.6504588  0.16649446 1.2894028  1.4015166
 1.5454441  1.8616961  0.6906408  1.6516988  0.37278974 1.164077
 1.2231334  0.15741226 1.1132643  1.5226622  0.9051585  0.23722242
 0.8946502  0.9433371  0.10974066 2.379972   0.26681584 1.0035309
 1.2899628  1.2486658  0.7818293  3.08416    0.3763667  2.3308012
 1.4034917  3.851936   0.148507   0.3579618  0.36321077 0.87803525
 2.975097   0.9489772  1.0255162  1.6400236  0.9158442  2.0193188
 1.6604759  1.3780726  0.92286223 1.5023596  1.3495436  1.4042667
 0.5571917  0.6391952  0.9395337  2.608418   0.82595944 1.5852743
 1.6700249  3.7086952  1.6135485  0.29979247 1.0988683  0.36580974
 1.4941156  0.76702857 3.0354836  2.6179283  0.5050524  1.1806972
 1.9157499  1.3361522  0.7964353  0.38774103 0.71324426 1.0127965
 0.1765697  1.2647655  

1/1 [==============================] - 0s 47ms/step
[2.9110658  1.7773309  1.4364505  0.7711489  0.9976975  1.6940836
 1.2735283  1.5121785  0.69709146 0.6331021  0.80458367 0.7462532
 0.39356846 0.79195356 0.58048654 0.13432899 1.1565003  1.2720279
 1.3991386  1.6878377  0.6101056  1.4903272  0.33274567 1.0357941
 1.0784491  0.1091986  0.9941584  1.3739473  0.79251    0.20630053
 0.7691052  0.83799326 0.11175942 2.1059484  0.22579269 0.87419564
 1.1556505  1.112772   0.6897795  2.73301    0.33502376 2.0410476
 1.2351371  3.3650088  0.15029833 0.30898726 0.34518752 0.7692941
 2.6453147  0.8680637  0.92182034 1.4390097  0.8074268  1.7685884
 1.4649066  1.2186729  0.8294058  1.3598639  1.2147132  1.2522193
 0.4969389  0.5707784  0.83717024 2.275015   0.7565446  1.3896238
 1.496927   3.2689216  1.4157077  0.26565123 0.90811557 0.35532174
 1.2878169  0.67374855 2.6423838  2.3155704  0.4468848  1.0255055
 1.742639   1.1575754  0.72126853 0.340549   0.6577408  0.91224796
 0.12846683 1.093253

1/1 [==============================] - 0s 26ms/step
[3.0596313  1.9470913  1.5528467  0.84027904 1.0661641  1.7355912
 1.3272284  1.6148136  0.7013544  0.6555347  0.8241271  0.77453935
 0.4141566  0.8772108  0.6044626  0.14848539 1.1966667  1.3140624
 1.4559549  1.7164938  0.6520746  1.5387622  0.3329497  1.0882022
 1.1636992  0.1283897  1.059286   1.4075539  0.83984464 0.20890789
 0.85111785 0.88523424 0.11330296 2.1890335  0.25156155 0.94876736
 1.2236518  1.1547759  0.6932772  2.8788521  0.35432976 2.2022526
 1.3201847  3.5633605  0.16035411 0.34022442 0.3438373  0.82129115
 2.7245514  0.87722605 0.9520226  1.5261524  0.84875697 1.8830255
 1.5732558  1.3091007  0.8654108  1.3680798  1.2675812  1.3151385
 0.54417884 0.58299273 0.8738066  2.4705906  0.77215534 1.4681337
 1.5284454  3.4346824  1.5225011  0.28771186 1.009295   0.36008063
 1.4080182  0.7028802  2.8566964  2.4457269  0.45638338 1.12759
 1.7865326  1.2504498  0.7221047  0.35759953 0.6801044  0.9257359
 0.16141829 1.1730342

1/1 [==============================] - 0s 33ms/step
[3.2778654  2.1643217  1.5992161  0.86546105 1.1488036  1.8790783
 1.4442967  1.6978266  0.8049719  0.682566   0.86629176 0.86094946
 0.4337415  0.93518394 0.66312265 0.16326275 1.2895347  1.4369218
 1.5616701  1.8316475  0.67305577 1.6320546  0.36160576 1.1656301
 1.2409972  0.16530642 1.1428372  1.4986448  0.9117617  0.23967414
 0.91003466 0.97578055 0.11796009 2.3561296  0.24842712 0.9753452
 1.3371329  1.285564   0.7507096  3.2114153  0.3874619  2.3312817
 1.4420991  3.891248   0.12714162 0.39906684 0.34695533 0.9030603
 2.934544   0.8883422  1.0298026  1.6535141  0.8855076  2.0379167
 1.651617   1.3842702  0.938645   1.41934    1.3295113  1.3897455
 0.5647765  0.6125285  1.0356195  2.677968   0.8405152  1.6060266
 1.6445363  3.726498   1.6136647  0.32655296 1.1505781  0.32809132
 1.4499843  0.75513405 3.0763748  2.6755273  0.49572363 1.2341552
 1.8580654  1.3487313  0.7901347  0.39260748 0.76243484 0.98160577
 0.17253564 1.328842

1/1 [==============================] - 0s 50ms/step
[3.2379785  2.1022105  1.6017768  0.8677643  1.1249186  1.8713332
 1.4238511  1.6834309  0.78663456 0.70549357 0.874005   0.8445479
 0.4375505  0.9134359  0.6605426  0.16627352 1.2726421  1.4077448
 1.5441507  1.8209234  0.67034185 1.6097013  0.38048592 1.1592448
 1.2131761  0.17119768 1.1197997  1.4952308  0.904718   0.24003527
 0.89873856 0.9530517  0.13240093 2.3349323  0.25217158 0.9875711
 1.302734   1.2616268  0.7804493  3.1202     0.38902053 2.3119893
 1.4181294  3.826936   0.15738156 0.37458286 0.36323178 0.88722515
 2.904297   0.90976655 1.0077324  1.6357265  0.8889641  2.013071
 1.6392967  1.3686178  0.9303836  1.4423997  1.320141   1.3868933
 0.5506684  0.6179062  0.9807463  2.62109    0.8178378  1.5700653
 1.627823   3.6692379  1.5984591  0.30827165 1.1158873  0.34686583
 1.4546664  0.7536775  3.0407736  2.6234565  0.48989123 1.1946956
 1.8648605  1.3401017  0.7693737  0.38536534 0.73382884 0.98505414
 0.1747667  1.2841444

In [36]:
from scipy import stats
def stat_cor(pred_arr, labels_arr):
    for idx, gene in enumerate(gene_dict.keys()):
        X1 = [row[idx] for row in labels_arr]
        X2 = [row[idx] for row in pred_arr]
        r,p = stats.pearsonr(X1, X2)
        print('%s,%s,%s,%s' %(gene, gene_dict[gene], r, p))
    

In [33]:
stat_cor(pred_arr, labels_arr)

ENSG00000007171,NOS2,0.038565925348557406,0.7374510810998474
ENSG00000021826,CPS1,0.048234448279873374,0.6749518698944225
ENSG00000026559,KCNG1,-0.1671986645599116,0.14342932069432854
ENSG00000057468,MSH4,-0.012403446067198747,0.9141703767676479
ENSG00000058404,CAMK2B,-0.09580267985087228,0.40407397135289497
ENSG00000064655,EYA2,-0.1380966129573413,0.22792084228550988
ENSG00000066405,CLDN18,0.1842450651627889,0.10635531461517682
ENSG00000075461,CACNG4,-0.01281375930478627,0.9113423805880438
ENSG00000080166,DCT,-0.08900543252237861,0.43838503044221666
ENSG00000081051,AFP,0.1490726355053066,0.19270805580638023
ENSG00000084674,APOB,-0.20197908048704308,0.07616960156979237
ENSG00000088726,TMEM40,0.1508058141377538,0.18752740493857795
ENSG00000092345,DAZL,0.07836796922202786,0.4952372476676015
ENSG00000093134,VNN3,0.044938849298440046,0.696034650403799
ENSG00000095777,MYO3A,0.015335408711345738,0.8939879190658551
ENSG00000096395,MLN,0.027133240776382978,0.813581063041973
ENSG00000099769,IGF

In [27]:
stat_cor(pred_min_arr, labels_arr)

ENSG00000007171,NOS2,-0.1049981693426,0.3602549884946787
ENSG00000021826,CPS1,-0.11130957682399312,0.33194325172269945
ENSG00000026559,KCNG1,-0.002071312907756116,0.9856404433845345
ENSG00000057468,MSH4,-0.14268455627200724,0.2126916693434342
ENSG00000058404,CAMK2B,0.1733967506826257,0.12896404613496962
ENSG00000064655,EYA2,0.08058825353877085,0.4830614388184009
ENSG00000066405,CLDN18,-0.013154018015712346,0.9089980532449403
ENSG00000075461,CACNG4,0.026805111296778737,0.8157955425478864
ENSG00000080166,DCT,0.09332377184430254,0.4164004249704256
ENSG00000081051,AFP,-0.10658422871474701,0.3530042262785222
ENSG00000084674,APOB,0.0752461582907521,0.5126249027960607
ENSG00000088726,TMEM40,-0.024694124171241998,0.8300759400905141
ENSG00000092345,DAZL,0.032540882554795286,0.777308534930123
ENSG00000093134,VNN3,-0.024840233288341333,0.8290857109756548
ENSG00000095777,MYO3A,0.19441483439635657,0.08807894132067783
ENSG00000096395,MLN,0.003165068653882565,0.9780594364547408
ENSG00000099769,IGFALS

In [46]:
stat_cor(pred_mean_arr, labels_arr)

ENSG00000007171,NOS2,-0.08136791273242469,0.4788239877853628
ENSG00000021826,CPS1,0.015732413169323174,0.8912599744815349
ENSG00000026559,KCNG1,-0.0022785845313266634,0.9842036904997141
ENSG00000057468,MSH4,-0.05611202597249186,0.6255840301604658
ENSG00000058404,CAMK2B,0.05348408853884226,0.6418845460132302
ENSG00000064655,EYA2,0.12627499003180262,0.27061747217949406
ENSG00000066405,CLDN18,0.15157504497052365,0.18526066996226623
ENSG00000075461,CACNG4,0.08711137092178371,0.4482301708606641
ENSG00000080166,DCT,-0.027709134928819025,0.8096980088359003
ENSG00000081051,AFP,0.052376385138938095,0.6488069750682669
ENSG00000084674,APOB,-0.20679662559598094,0.06928115956775262
ENSG00000088726,TMEM40,0.07696408975008252,0.5030180671002016
ENSG00000092345,DAZL,0.13479420258844155,0.23934369813890088
ENSG00000093134,VNN3,0.02288189463423899,0.8423795290104005
ENSG00000095777,MYO3A,0.11662254335703796,0.30923524051756923
ENSG00000096395,MLN,0.08275231527519461,0.4713490485394742
ENSG00000099769,IG

In [49]:
stat_cor(pred_quantile_arr, labels_arr)

ENSG00000007171,NOS2,-0.00042727658839101807,0.9970377178745256
ENSG00000021826,CPS1,0.046562781922578136,0.6856159476309227
ENSG00000026559,KCNG1,0.015038488682477648,0.8960289569420999
ENSG00000057468,MSH4,0.03748392434597125,0.7445614774119119
ENSG00000058404,CAMK2B,0.019389910973877725,0.8661914639782119
ENSG00000064655,EYA2,0.044902959681714545,0.6962655531677766
ENSG00000066405,CLDN18,0.17343885725387087,0.12886972678023953
ENSG00000075461,CACNG4,0.026670870467643243,0.8167019235949223
ENSG00000080166,DCT,-0.08900008816761489,0.4384126373089254
ENSG00000081051,AFP,0.030359743059869257,0.7918863974841348
ENSG00000084674,APOB,-0.24737845702426017,0.028993578853101054
ENSG00000088726,TMEM40,0.15047390816598663,0.18851162272823313
ENSG00000092345,DAZL,0.10869812578766648,0.3434821879593246
ENSG00000093134,VNN3,-0.013815661763889122,0.9044416758887885
ENSG00000095777,MYO3A,0.04413753262318597,0.701196614884642
ENSG00000096395,MLN,0.06065398473455846,0.59783326314781
ENSG00000099769,IG

### 已经没用的

In [24]:
BATCH_SIZE = 8000

genes_arr, pred_arr = [], []
features = []
files = ['TCGA-5M-AAT4-01Z-00-DX1.725C46CA-9354-43AC-AA81-3E5A66354D6B.txt', \
             'TCGA-A6-2671-01Z-00-DX1.13d1a0d9-78cd-4cfc-b670-34a79ebe52ee.txt', \
              'TCGA-A6-2683-01Z-00-DX1.0dfc5d0a-68f4-45e1-a879-0428313c6dbc.txt', \
              'TCGA-A6-6648-01Z-00-DX1.88b9a490-0bed-43f3-bd74-1bf2810f6884.txt', \
              'TCGA-AA-3494-01Z-00-DX1.E275AF20-3AA7-4191-BD1F-FFE744CA6A2F.txt', \
              'TCGA-AA-3662-01Z-00-DX1.625F1BCC-5E59-411E-AE23-6F43CE6122B2.txt', \
              'TCGA-AA-3680-01Z-00-DX1.9eef1b8f-c3c1-486f-83e7-a88182ce892a.txt', \
              'TCGA-AA-3679-01Z-00-DX1.b3445f8e-b143-4f24-9edd-8abdcb6b139b.txt' ]
for file_name in files:
    gene = get_gene_from_dict(file_name)
    genes_arr.append(gene)
    
    file_path = os.path.join(r'D:\项目资料\基因表达\数据', file_name)
    features = get_features(file_path)
    results = new_model.predict(tf.convert_to_tensor(features), batch_size=len(features), verbose=1)
    x = agregate(results)
    pred_arr.append(x)
    


1/1 [==============================] - 0s 49ms/step


In [17]:
import numpy as np

import pandas as pd

# pct_chg = pd.Series({'000001.SZ':-1.7391, '000002.SZ':0.6250,'000004.SZ':1.1378,'002600.SZ':0.0000,'000010.SZ':-1.0013})
# vol = pd.Series({'000001.SZ':249326.31,'000002.SZ':338224.97,'000004.SZ':211876.00,'000010.SZ':222782.00,'002600.SZ':342096.76})
# pccs = np.corrcoef(pct_chg, vol)
a = np.array([-1.7391, 0.6250, 1.1378, -1.0013, 0.0000])
b = np.array([249326.31, 338224.97, 211876.00, 222782.00, 342096.76])
pccs = np.corrcoef(a, b)

print(pccs)

[[ 1.         -0.19247453]
 [-0.19247453  1.        ]]


In [14]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

pct_chg = pd.Series({'000001.SZ':-1.7391, '000002.SZ':0.6250,'000004.SZ':1.1378,'002600.SZ':0.0000,'000010.SZ':-1.0013})
vol = pd.Series({'000001.SZ':249326.31,'000002.SZ':338224.97,'000004.SZ':211876.00,'000010.SZ':222782.00,'002600.SZ':342096.76})

pccs = pearsonr(pct_chg, vol)

print(pccs)


(-0.19247452749413846, 0.7564554804046555)


In [6]:
import os

# 读取特征数据
for file_name in os.listdir(r'D:\项目资料\基因表达\数据'):
    file_path = os.path.join(r'D:\项目资料\基因表达\数据', file_name)
    if os.path.isfile(file_path) and file.endswith('.txt'):
        print('file: %s' % file_name)
        labels = get_gene_from_dict(file_name)
        features = get_features(file_path)
        print(len(labels))
        print(len(features))


file: TCGA-5M-AAT4-01Z-00-DX1.725C46CA-9354-43AC-AA81-3E5A66354D6B.txt
537
8000
file: TCGA-A6-2671-01Z-00-DX1.13d1a0d9-78cd-4cfc-b670-34a79ebe52ee.txt
537
8000
file: TCGA-A6-2683-01Z-00-DX1.0dfc5d0a-68f4-45e1-a879-0428313c6dbc.txt
537
8000
file: TCGA-A6-6648-01Z-00-DX1.88b9a490-0bed-43f3-bd74-1bf2810f6884.txt
537
5563
file: TCGA-AA-3494-01Z-00-DX1.E275AF20-3AA7-4191-BD1F-FFE744CA6A2F.txt
537
17
file: TCGA-AA-3662-01Z-00-DX1.625F1BCC-5E59-411E-AE23-6F43CE6122B2.txt
537
41
file: TCGA-AA-3679-01Z-00-DX1.b3445f8e-b143-4f24-9edd-8abdcb6b139b.txt
537
8000
file: TCGA-AA-3680-01Z-00-DX1.9eef1b8f-c3c1-486f-83e7-a88182ce892a.txt
537
8000
